In [1]:
import numpy as np
from numpy import random
import pandas as pd
import matplotlib.pyplot as plt
import math, decimal
from math import exp
import seaborn as sns
import sklearn as sk
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

In [20]:
train = pd.read_csv("train.csv", index_col = None)
y = train[['LABELS']]
x = train.drop('LABELS', axis=1)
xMean = x.mean()
test_nolabels = pd.read_csv("test_nolabels.csv", index_col = None)

In [7]:
print(np.shape(x))

(62000, 217)


In [22]:
# Splitting trainig/Validation
x_train, x_validation, y_train, y_validation = train_test_split( x,y, test_size=0.2)
## Replacing possible missing values
x_train = x_train.fillna(xMean)
x_validation = x_validation.fillna(xMean)

In [19]:
## Data eploration
print("",np.shape(x_train),"  :",np.shape(x_validation) )
# print(y_train['LABELS'].value_counts())

print("Label balance on Training set: ", "\n", y_train['LABELS'].value_counts())
print("Label balance on Validation set: ", "\n", y_validation['LABELS'].value_counts())


 (49600, 217)   : (12400, 217)
Label balance on Training set:  
 1.0    33030
0.0    16570
Name: LABELS, dtype: int64
Label balance on Validation set:  
 1.0    8187
0.0    4213
Name: LABELS, dtype: int64


In [ ]:
for i in x_train.columns:
    print(i)

In [ ]:
# To adapt the prediction to Kaggel format
def formating_prediction(predictions): 
        '''
        return de predicted classes from the hypotesis function result (sigmoid(W,X))
        @hypotesis : matrix of probablilities 
        '''
        y_hat = pd.DataFrame({'S.No' : [],'LABELS' : []}, dtype=np.int8) 
        for i in range(len(predictions)):
            y_hat.loc[i] = [i,predictions[i]]
       
        return pd.DataFrame(data = y_hat) 

In [12]:
# feature extraction based on domain knowledge 
ndvi_list = {'NDVI_jan','NDVI_feb','NDVI_mar','NDVI_apr','NDVI_may','NDVI_jun','NDVI_jul','NDVI_aug','NDVI_sep','NDVI_oct','NDVI_nov','NDVI_dec'}
## Extrae los NDVI y escatter contra las labels

In [ ]:
## Adapting prediction to Kaggel needs
def formating_prediction(predictions): 
        '''
        return de predicted classes from the hypotesis function result (sigmoid(W,X))
        @hypotesis : matrix of probablilities 
        '''
        y_hat = pd.DataFrame({'S.No' : [],'LABELS' : []}, dtype=np.int8) 
        for i in range(len(predictions)):
            y_hat.loc[i] = [i,predictions[i]]
       
        return pd.DataFrame(data = y_hat) 

In [ ]:
## XGBOOST
def xgboost(X_train, y_train, X_test):    
    # fit model no training data
    model = XGBClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    print("Accuracy xgboost: %.2f%%" % (accuracy * 100.0))
    predictionsDF = formating_prediction(predictions)
    return model, accuracy, predictionsDF

In [ ]:
# create grill for RF training
rf_training_gril = {
    'n_estimators': np.linspace(10, 200).astype(int),
    'max_depth': [None] + list(np.linspace(3, 20).astype(int)),
    'max_features': ['auto', 'sqrt', None] + list(np.arange(0.5, 1, 0.1)),
    'max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int)),
    'min_samples_split': [2, 5, 10],
    'bootstrap': [True, False]
}

# Create the random search model
rs = RandomizedSearchCV(estimator, rf_training_gril, n_jobs = -1, 
                        scoring = 'roc_auc', cv = 3, 
                        n_iter = 10, verbose = 1, random_state=RSEED)

# Random Forest
def randomForest(x_train, y_train, x_validation, y_validation, estimators):
    '''
    Performe random forest, in the simples way, from Skl library
    @estimators: NUmber of estimators
    return: model "classifier_LR" and a prediction over a x_validation
    '''
    rf_classifier = RandomForestClassifier(n_estimators = estimators, criterion = 'entropy', random_state = 42)
    rf_classifier.fit(x_train, y_train)

    # Predicting the over the validation set
    y_pred = rf_classifier.predict(x_validation)
    print(error(y_pred,y_validation))

    prediction = rf_classifier.predict(test_Set)
    df = pd.DataFrame(prediction)
    df.to_csv("rf_prediction.csv", index = None)
    accuracy = accuracy_score(y_test, predictions)
    
    # Uncomment to save the model
    # joblib.dump(rf_classifier, 'randomforestmodel.pkl')
    
    predictionsDF = formating_prediction(prediction)
    return rf_classifier, accuracy, predictionsDF 

        # # load the model from disk to predict new dataSet
        # loaded_model = pickle.load(open(filename, 'rb'))
        # result = loaded_model.score(X_test, Y_test)

def evaluate_Rf(X_train, y_train, X_test, y_test, rf_classifier):
    features = X_train.columns
    train_rf_predictions = rf_classifier.predict(X_train)
    train_rf_probs = rf_classifier.predict_proba(X_train)[:, 1]

    rf_predictions = rf_classifier.predict(X_test)
    rf_probs = rf_classifier.predict_proba(X_test)[:, 1]
    
    evaluate_model(rf_predictions, rf_probs, train_rf_predictions, train_rf_probs)
    cm = confusion_matrix(y_test, rf_predictions)
    plot_confusion_matrix(cm, classes = ['Cropland', 'Non-Cropland'],
                      title = 'Cropland Confusion Matrix')
    fi_model = pd.DataFrame({'feature': features,
                   'importance': rf_classifier.feature_importances_}).\
                    sort_values('importance', ascending = True)
     
    return fi_model


    

In [ ]:
# predict and format prediction


In [ ]:
# Save model and sve predisction
